In [3]:
import numpy as np
import pandas as pd
from survONS import SurvONS
from utils import date_discretization
import matplotlib.pyplot as plt
import random

p = 0.05
df = pd.read_csv(
         "/home/espe/pySurvONS/ventasNIC_censurado2.csv",
         header=0, 
         skiprows=lambda i: i>0 and random.random() > p
)

In [ ]:
i_dates_df = df[["anho", "mes", "dia"]].rename(columns={"anho": "year", "mes": "month", "dia": "day"})
f_dates_df = df[["anho_renov", "mes_renov", "dia_renov"]].rename(columns={"anho_renov": "year", "mes_renov": "month", "dia_renov": "day"})
dates_df = pd.concat([i_dates_df, f_dates_df])
dates=date_discretization(pd.to_datetime(dates_df[["year", "month", "day"]]).to_numpy())
print(dates)



In [ ]:
df.insert(0, "initial_time", dates[0:df.shape[0]])
df.insert(19,"final_time", dates[df.shape[0]:(df.shape[0]*2)])
df.drop(labels=["anho","mes","dia","anho_renov","mes_renov","dia_renov", "pais_compra"], axis=1, inplace=True)
df.head()

In [ ]:
df["pais"] = pd.Categorical(df["pais"]).codes
df["moneda"] = pd.Categorical(df["moneda"]).codes
df["monto"] = df["monto"].apply(lambda x: x/10000)
df["nac_titular"] = df["nac_titular"].apply(lambda x: x/1000)

df.head()


In [7]:
df.drop(labels=["tiempo", "time", "tiempo_tarifa", "nro_duenhos", "nro_trans"], axis=1, inplace=True)

In [ ]:
x = df.drop(labels=["final_time", "cens"], axis=1)

surv = SurvONS()
surv.train(x, df["initial_time"].to_numpy(), df["final_time"].to_numpy(), df["cens"].to_numpy(), diam=2)

In [ ]:
X = x.to_numpy()
np.random.shuffle(X)
surv.plot(X[0:100], 0, 5000)

In [ ]:
#surv.iterative_train(x, df["initial_time"].to_numpy(), df["final_time"].to_numpy(), df["cens"].to_numpy())
#surv.score(df["final_time"].to_numpy(), x.to_numpy(), df["cens"].to_numpy())
surv.predict_time(x.to_numpy()[0], 1)
print(surv.t_max)
#surv.predict(x.to_numpy()[0], 1000, 1)